## *Vampire* 5th edition

[Official site.](https://www.worldofdarkness.com/vampire-the-masquerade)

This edition works as follows:

1. Roll a pool of d10s. Some of these will be normal dice, and some will be Hunger dice.
2. Count each 6+ as a success.
3. For each pair of 10s, add two additional successes (for a total of four from those two dice).
4. If the total number of successes meets or exceeds the difficulty, you win. Otherwise you lose.

In addition to the binary win/lose aspect of the outcome, there are the following special rules:

* If you won and had at least one pair of 10s, you score a **critical win**.
* However, if you scored a critical win and at least one Hunger die shows a 10, it becomes a **messy critical** instead.
* If you lost and at least one Hunger die shows a 1, you score a **bestial failure**.

## Code

This example will demonstrate a couple features of `hdroller`:

* `EvalPool` with multiple pools. Each pool will generate an independent `count`.
* Multivariate dice: Dice will have three axes, namely the number of successes, the win type, and the lose type.

In [1]:
try:
    # JupyterLite
    import micropip
    await micropip.install('hdroller')
except ModuleNotFoundError:
    # Jupyter
    import sys
    !{sys.executable} -m pip install hdroller

import hdroller

We start by translating the V5 dice mechanic into an `EvalPool`.

In [2]:
class EvalVampire5(hdroller.EvalPool):
    def next_state(self, prev_state, outcome, normal, hunger):
        if prev_state is None:
            successes, win_type, lose_type = 0, '', ''
        else:
            successes, win_type, lose_type = prev_state
        if outcome == 'crit':
            total_crit = normal + hunger
            # Crits count as successes, and every pair adds 2 more.
            successes += total_crit + 2 * (total_crit // 2)
            if total_crit >= 2:
                if hunger > 0:
                    win_type = 'messy'
                else:
                    win_type = 'crit'
        elif outcome == 'success':
            successes += normal + hunger
        elif outcome == 'botch':
            if hunger > 0:
                lose_type = 'bestial'
        else:  # normal loss
            pass

        return successes, win_type, lose_type

v5_eval = EvalVampire5()

Along with this, the V5 die:

In [3]:
v5_die = hdroller.Die({'botch' : 1, 'failure' : 4, 'success' : 4, 'crit' : 1})

Now we can construct the normal and Hunger pools and evaluate:

In [4]:
result = v5_eval(v5_die.pool(3), v5_die.pool(2))
print(result)

Total weight: 100000
| Outcome[0] | Outcome[1] | Outcome[2] | Weight | Probability |
|-----------:|-----------:|-----------:|-------:|------------:|
|          0 |            |            |   2000 |   2.000000% |
|          0 |            |    bestial |   1125 |   1.125000% |
|          1 |            |            |  11000 |  11.000000% |
|          1 |            |    bestial |   4625 |   4.625000% |
|          2 |            |            |  23160 |  23.160000% |
|          2 |            |    bestial |   6840 |   6.840000% |
|          3 |            |            |  23632 |  23.632000% |
|          3 |            |    bestial |   4368 |   4.368000% |
|          4 |            |            |  11776 |  11.776000% |
|          4 |            |    bestial |   1024 |   1.024000% |
|          4 |       crit |            |    240 |   0.240000% |
|          4 |       crit |    bestial |    135 |   0.135000% |
|          4 |      messy |            |    725 |   0.725000% |
|          4 |     

Or, we can bind the die to the evaluator and just provide the pool sizes:

In [5]:
v5_eval_bound = v5_eval.bind_dice(v5_die, v5_die)
result = v5_eval_bound(3, 2)
print(result)

Total weight: 100000
| Outcome[0] | Outcome[1] | Outcome[2] | Weight | Probability |
|-----------:|-----------:|-----------:|-------:|------------:|
|          0 |            |            |   2000 |   2.000000% |
|          0 |            |    bestial |   1125 |   1.125000% |
|          1 |            |            |  11000 |  11.000000% |
|          1 |            |    bestial |   4625 |   4.625000% |
|          2 |            |            |  23160 |  23.160000% |
|          2 |            |    bestial |   6840 |   6.840000% |
|          3 |            |            |  23632 |  23.632000% |
|          3 |            |    bestial |   4368 |   4.368000% |
|          4 |            |            |  11776 |  11.776000% |
|          4 |            |    bestial |   1024 |   1.024000% |
|          4 |       crit |            |    240 |   0.240000% |
|          4 |       crit |    bestial |    135 |   0.135000% |
|          4 |      messy |            |    725 |   0.725000% |
|          4 |     

We can use the `[]` operator to marginalize dimensions from the result.
For example, dimension 1 is the win type.

In [6]:
print(result[1])

Total weight: 100000
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|         |  91854 |  91.854000% |
|    crit |   2268 |   2.268000% |
|   messy |   5878 |   5.878000% |

